Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [165]:
import heapq
import json
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

In [166]:
PUZZLE_DIM = 3
RANDOMIZE_STEPS = 100_000
FINAL_STATE = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

action = namedtuple('Action', ['pos1', 'pos2'])

## Helper functions

In [167]:
def available_actions(state: np.ndarray, preferencial_dir=None) -> list["Action"]:
    x, y = [int(_[0]) for _ in np.where(state == 0)]  
    actions = []  
    
    # move definition
    if x > 0:  # up
        actions.append(action((x, y), (x - 1, y)))
    if y < PUZZLE_DIM - 1:  # right
        actions.append(action((x, y), (x, y + 1)))
    if y > 0:  # left
        actions.append(action((x, y), (x, y - 1)))
    if x < PUZZLE_DIM - 1:  # down
        actions.append(action((x, y), (x + 1, y)))
    
    # preferencial directions
    direction_order = {
        "up": ["up", "right", "left", "down"],
        "down": ["down", "right", "left", "up"],
        "right": ["right", "down", "up", "left"],
        "left": ["left", "down", "up", "right"],
        None: ["left","up","right", "down"], 
    }
    
    # Map actions and strings if they are present
    action_map = {
        "up": action((x, y), (x - 1, y)) if x > 0 else None,
        "right": action((x, y), (x, y + 1)) if y < PUZZLE_DIM - 1 else None,
        "left": action((x, y), (x, y - 1)) if y > 0 else None,
        "down": action((x, y), (x + 1, y)) if x < PUZZLE_DIM - 1 else None,
    }
    
    # Ordina le azioni disponibili secondo il preferencial_dir
    sorted_actions = [
        action_map[dir] for dir in direction_order[preferencial_dir] if action_map[dir] is not None
    ]
    
    return sorted_actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def is_final_sol(state: np.ndarray) -> bool:
    return np.array_equal(state, FINAL_STATE)

## Random Initialization

In [168]:
curr_state = FINAL_STATE
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    curr_state = do_action(curr_state, choice(available_actions(curr_state)))
curr_state

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 122846.65it/s]


array([[0, 8, 5],
       [6, 3, 2],
       [7, 1, 4]])

## Move Functions

In [169]:
def distance_between(x1,y1,x2,y2):
    return  np.abs(x1 - x2) + np.abs(y1 - y2)

In [170]:
def actions_from_zero_to_point(state:np.ndarray,x,y):
    path = []
    current = state
    n = current[x,y]
    x0, y0 = np.where(current==0)
    while distance_between(x,y,x0,y0) > 1:
        for a in available_actions(current):
            next = do_action(current,a)
            nx0, ny0 = np.where(next==0)
            if distance_between(x,y,nx0,ny0)< distance_between(x,y,x0,y0) and current[nx0,ny0] > n:
                current = next
                path.append(current)
                print(current)
                x0, y0 = nx0, ny0
                break
    return path, current

In [171]:
def put_zero_below(state:np.ndarray, x, y, dir):
    path = []
    current = state
    n = current[x,y]
    x0, y0 = np.where(current==0)

    while distance_between(x+1,y,x0,y0) != 0:
        for a in available_actions(current,dir):
            next = do_action(current,a)
            nx0, ny0 = np.where(next==0)
            if distance_between(x+1,y,nx0,ny0)< distance_between(x+1,y,x0,y0) and next[x,y]==n:
                current = next
                path.append(current)
                print(current)
                x0, y0 = nx0, ny0
                break
    return path, current

In [172]:
def slide_row(state:np.ndarray):
    current = state
    path = []
    x0, y0 = np.where(current==0)
    d_x, d_y = x0 -1 , len(FINAL_STATE)-1 
    while distance_between(d_x,d_y,x0,y0) != 0:
        for a in available_actions(current,"up"):
            next = do_action(current,a)
            nx0, ny0 = np.where(next==0)
            if distance_between(d_x,d_y,nx0,ny0)< distance_between(d_x,d_y,x0,y0):
                current = next
                path.append(current)
                print(current)
                x0, y0 = nx0, ny0
                break
    return path, current
    

In [173]:
def slide_row_back(state:np.ndarray):
    current = state
    path = []
    x0, y0 = np.where(current==0)
    d_x, d_y = x0 + 1 , 0
    while distance_between(d_x,d_y,x0,y0) != 0:
        for a in available_actions(current,"left"):
            next = do_action(current,a)
            nx0, ny0 = np.where(next==0)
            if distance_between(d_x,d_y,nx0,ny0)< distance_between(d_x,d_y,x0,y0):
                current = next
                path.append(current)
                print(current)
                x0, y0 = nx0, ny0
                break
    return path, current

In [174]:
def put_last_right(state:np.ndarray):
    current = state
    path = []
    x,y = np.where(current==0)
    move_set= [
        action((x, y), (x + 1, y)), #down
        action((x + 1, y), (x + 1, y - 1)), #left
        action((x + 1, y - 1), (x, y - 1)) #up
    ]
    for act in move_set:
        current = do_action(current,act)
        print(current)
        path.append(current)
        
    return path, current

In [175]:
def action_move_right_col(state:np.ndarray,x,y):
    '''moves a number to the correct y position'''
    path = []
    current = state
    n = current[x,y]
    x0, y0 = np.where(current==0)
    x_f, y_f = np.where(FINAL_STATE==n) 
    if y_f == len(FINAL_STATE)-1:
        x_f+=1
    prev_x0,prev_y0 = x0,y0
    while y != y_f:
        if y<y_f:
            # move n right
            while (y0,x0)!=(y+1,x):
                same_line = x0==x
                for a in available_actions(current,"right"):
                    next = do_action(current,a)
                    nx0, ny0 = np.where(next==0)
                    if next[x,y]==n and np.abs(y-ny0) < 2 and np.abs(x-nx0) < 2 and (nx0,ny0)!=(prev_x0,prev_y0):
                        if ( ( same_line and np.abs(ny0-(y+1))<=np.abs(y0-(y+1)) ) or ( not same_line and distance_between(nx0,ny0,x,y+1)<distance_between(x0,y0,x,y+1) ) ):
                            prev_x0, prev_y0 = x0,y0
                            current = next
                            path.append(current)
                            print(current)

                            x0, y0 = nx0, ny0
                            break
            left = action((x0, y0), (x0, y0 - 1))
            current = do_action(current,left)
            path.append(current)
            print(current)

            x,y = np.where(current==n)
            x0, y0 = np.where(current==0)
        else:
            # move n left
            while (y0,x0)!=(y-1,x):
                same_line = x0==x
                for a in available_actions(current,"left"):
                    next = do_action(current,a)
                    nx0, ny0 = np.where(next==0)
                    if next[x,y]==n and np.abs(y-ny0) < 2 and np.abs(x-nx0) < 2 and (nx0,ny0)!=(prev_x0,prev_y0):
                        if ( ( same_line and np.abs(ny0-(y-1))<=np.abs(y0-(y-1)) ) or ( not same_line and distance_between(nx0,ny0,x,y-1)<distance_between(x0,y0,x,y-1) ) ):
                            prev_x0, prev_y0 = x0,y0
                            current = next
                            path.append(current)
                            print(current)

                            x0, y0 = nx0, ny0
                            break
            right = action((x0, y0), (x0, y0 + 1))
            current = do_action(current,right)
            print(current)

            path.append(current)
            x,y = np.where(current==n)
            x0, y0 = np.where(current==0)
    return path, current

In [176]:
def action_move_right_row(state:np.ndarray,x,y):
    '''moves a number to the correct x position'''
    path = []
    current = state
    n = current[x,y]
    x0, y0 = np.where(current==0)
    x_f, y_f = np.where(FINAL_STATE==n) 
    if y_f == len(FINAL_STATE)-1:
        x_f +=1
    prev_x0,prev_y0 = x0,y0
    while x != x_f:
        if x<x_f:
            # move n down
            while (y0,x0)!=(y,x+1):
                same_col = y0==y
                for a in available_actions(current, "down"):
                    next = do_action(current,a)
                    nx0, ny0 = np.where(next==0)
                    if next[x,y]==n and np.abs(y-ny0) < 2 and np.abs(x-nx0) < 2 and (nx0,ny0)!=(prev_x0,prev_y0):
                        if ( ( same_col and np.abs(nx0-(x+1)) <= np.abs(x0-(x+1)) ) or ( not same_col and distance_between(nx0,ny0,x+1,y)<distance_between(x0,y0,x+1,y)  ) ):
                            prev_x0, prev_y0 = x0,y0
                            current = next
                            path.append(current)
                            x0, y0 = nx0, ny0
                            break
            up = action((x0, y0), (x0 - 1, y0))
            current = do_action(current,up)
            path.append(current)
            x,y = np.where(current==n)
            x0, y0 = np.where(current==0)
        else:
            # move n up
            if ((x,y)==(x_f+1,y_f) and (x0,y0)!=(x_f+1,y_f) and n>1 and y0 < y):
                moves, current = put_zero_below(current,x,y,"right")
                path.extend(moves)
                x,y = np.where(current==n)
                x0, y0 = np.where(current==0)
            while (y0,x0)!=(y,x-1):
                same_col = y0==y
                for a in available_actions(current, "up"):
                    next = do_action(current,a)
                    nx0, ny0 = np.where(next==0)
                    if next[x,y]==n and np.abs(y-ny0) < 2 and np.abs(x-nx0) < 2 and (nx0,ny0)!=(prev_x0,prev_y0) :
                        if ( ( same_col and np.abs(nx0-(x-1)) <= np.abs(x0-(x-1)) ) or ( not same_col and distance_between(nx0,ny0,x-1,y)<distance_between(x0,y0,x-1,y)  ) ):
                            prev_x0, prev_y0 = x0,y0
                            current = next
                            path.append(current)
                            print(current)
                            x0, y0 = nx0, ny0
                            break
            down = action((x0, y0), (x0 + 1, y0))
            current = do_action(current,down)
            path.append(current)
            print(current)
            x,y = np.where(current==n)
            x0, y0 = np.where(current==0)
    return path, current

In [177]:
from dataclasses import dataclass

@dataclass
class Story_Item:
    score: int
    pattern : list

In [178]:
def manhattan_distance(state: np.ndarray) -> int:
    """Calcola la distanza di Manhattan per tutte le tessere."""
    distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:  # Ignora il vuoto
                target_x, target_y = np.where(FINAL_STATE==value)
                distance += abs(x - target_x) + abs(y - target_y)
    return distance

In [179]:
def manhattan_distance_point(state: np.ndarray, x, y) -> int:
    """Calcola la distanza di Manhattan per un punto."""
    distance = 0
    target_x, target_y = np.where(FINAL_STATE==state[x,y])
    distance += abs(x - target_x) + abs(y - target_y)
    return distance

In [180]:
def lead_n_to_pos(state:np.ndarray,n:int):
    current = state
    path = []
    x , y = np.where(current==n)
    x_f, y_f = np.where(FINAL_STATE==n) 
    # Move zero near to number to move
    if ((x,y)!=(x_f,y_f)):
        steps, current = actions_from_zero_to_point(current,x,y)
        path.extend(steps)
        x , y = np.where(current==n)
    # move the number to right col
    steps, current = action_move_right_col(current,x,y)
    path.extend(steps)
    x , y = np.where(current==n)
    # move number to right row
    steps, current = action_move_right_row(current,x,y)
    path.extend(steps)
    #print(path[-1])
    return path, current


In [181]:
#lead_n_to_pos(curr_state,1)

## Layer Solving

In [182]:
def solve_first_layer(state):
    current = state
    path = []
    path.append(current)
    # solve first n-1 number in the line
    for n in range(1,FINAL_STATE.shape[0]+1):
        moves_done, current = lead_n_to_pos(current,n)
        path.extend(moves_done)

    # move zero below 1 to be able to slide the row
    x1, y1 = np.where(current==1)
    moves_done, current = put_zero_below(current,x1,y1,"left")
    path.extend(moves_done)

    #slide the row in order to leave space for the last number
    moves_done, current = slide_row(current)
    path.extend(moves_done)

    # put last number right and zero to the left
    moves_done, current = put_last_right(current)
    path.extend(moves_done)

    #slide the row in order to leave space for the last number
    moves_done, current = slide_row_back(current)
    path.extend(moves_done)

    return path

In [183]:
sol = solve_first_layer(curr_state)

[[8 0 5]
 [6 3 2]
 [7 1 4]]
[[8 3 5]
 [6 0 2]
 [7 1 4]]
[[8 3 5]
 [0 6 2]
 [7 1 4]]
[[8 3 5]
 [7 6 2]
 [0 1 4]]
[[8 3 5]
 [7 6 2]
 [1 0 4]]
[[8 3 5]
 [7 0 2]
 [1 6 4]]
[[8 3 5]
 [0 7 2]
 [1 6 4]]
[[8 3 5]
 [1 7 2]
 [0 6 4]]
[[8 3 5]
 [1 7 2]
 [6 0 4]]
[[8 3 5]
 [1 0 2]
 [6 7 4]]
[[8 0 5]
 [1 3 2]
 [6 7 4]]
[[0 8 5]
 [1 3 2]
 [6 7 4]]
[[1 8 5]
 [0 3 2]
 [6 7 4]]
[[1 8 5]
 [3 0 2]
 [6 7 4]]
[[1 8 5]
 [3 2 0]
 [6 7 4]]
[[1 8 0]
 [3 2 5]
 [6 7 4]]
[[1 0 8]
 [3 2 5]
 [6 7 4]]
[[1 2 8]
 [3 0 5]
 [6 7 4]]
[[1 2 8]
 [0 3 5]
 [6 7 4]]
[[1 2 8]
 [6 3 5]
 [0 7 4]]
[[1 2 8]
 [6 3 5]
 [7 0 4]]
[[1 2 8]
 [6 3 5]
 [7 4 0]]
[[1 2 8]
 [6 3 0]
 [7 4 5]]
[[1 2 8]
 [6 0 3]
 [7 4 5]]
[[1 2 8]
 [0 6 3]
 [7 4 5]]
[[0 2 8]
 [1 6 3]
 [7 4 5]]
[[2 0 8]
 [1 6 3]
 [7 4 5]]
[[2 8 0]
 [1 6 3]
 [7 4 5]]
[[2 8 3]
 [1 6 0]
 [7 4 5]]
[[2 8 3]
 [1 0 6]
 [7 4 5]]
[[2 0 3]
 [1 8 6]
 [7 4 5]]
[[0 2 3]
 [1 8 6]
 [7 4 5]]
[[1 2 3]
 [0 8 6]
 [7 4 5]]


In [184]:
print(f"N moves: {len(sol)}")

N moves: 34


In [185]:
sol[-1]

array([[1, 2, 3],
       [0, 8, 6],
       [7, 4, 5]])

## A-Star solver

In [186]:
def a_star_solver(initial_state: np.ndarray) -> list[np.ndarray]:
    """Trova la soluzione del puzzle usando A*."""
    # Strutture dati per A*
    open_set = []
    heapq.heappush(open_set, (0, json.dumps(initial_state.tolist())))
    came_from = {}
    g_score = {json.dumps(initial_state.tolist()): 0}
    f_score = {json.dumps(initial_state.tolist()): manhattan_distance(initial_state)}

    visited = set()

    while open_set:
        _, current_str = heapq.heappop(open_set)
        current = np.array(json.loads(current_str))  # Convert string back to NumPy array
        visited.add(current_str)

        # Verifica se abbiamo trovato la soluzione
        if np.array_equal(current, FINAL_STATE):
            # Ricostruisci il percorso
            path = []
            while current_str in came_from:
                path.append(np.array(json.loads(current_str)))
                current_str = came_from[current_str]
            path.append(FINAL_STATE)
            return path[::-1]

        # Espandi i nodi
        for action in available_actions(current):
            neighbor = do_action(current, action)
            neighbor_str = json.dumps(neighbor.tolist())

            if neighbor_str in visited:
                continue

            tentative_g_score = g_score[current_str] + 1

            if neighbor_str not in g_score or tentative_g_score < g_score[neighbor_str]:
                came_from[neighbor_str] = current_str
                g_score[neighbor_str] = tentative_g_score
                f_score[neighbor_str] = tentative_g_score + manhattan_distance(neighbor)
                heapq.heappush(open_set, (f_score[neighbor_str], neighbor_str))

    return None 

In [187]:
#solution = a_star_solver(curr_state)
#print(solution)
#print(f"Mosse: {len(solution)}")